# Julio Patti Pereira 
- 14/02/2025

# Introdução

- Este é um trabalho complementar ao notebook "Explore_data_and_get_model_1.ipynb".
   - Leia ele para a contextualização

- Basicamente é uma adaptação para a abordagem como regressão linear.
   - Utiliza-se também do LightGBM, desta vez em sua versão de regressão
   - Metrica de otimização: Minimização da RMSE

- Não há grande rigor nesse estudo, o principal objetivo é, apenas, se ter um regressor que foi treinado sobre os mesmos dados, e nada a mais, que o preditor do notebook "Explore_data_and_get_model_1.ipynb".

# Imports de bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from lightgbm import LGBMRegressor
from skopt import gp_minimize
from sklearn.metrics import mean_squared_error
from skopt.space import Real, Categorical, Integer
import pickle

# Import train data

In [2]:
df_train = pd.read_csv('../data/df_train.csv')
df_train.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
count,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000
mean,8.264305,0.531463,0.267056,2.513937,0.087316,15.858786,46.559798,0.996671,3.313017,0.654802,10.432475,5.625575
std,1.707009,0.186630,0.193222,1.331599,0.046865,10.534171,33.126142,0.001839,0.154486,0.160925,1.073139,0.824693
min,4.600000,0.120000,0.000000,1.200000,0.012000,1.000000,6.000000,0.990200,2.880000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.069000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,38.000000,0.996680,3.310000,0.620000,10.100000,6.000000
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997800,3.400000,0.720000,11.100000,6.000000
max,15.900000,1.580000,0.760000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,1.950000,14.900000,8.000000


In [3]:
feature_columns = df_train.drop(columns=['quality']).columns.tolist()

def dist_quality(df, column='quality'):
    count = df[column].value_counts()
    prop = df[column].value_counts(normalize=True)
    return pd.concat([count, prop], axis=1)

In [4]:
dist_quality(df_train, column='quality')

,count,proportion
quality,,
5,461,0.424103
6,428,0.393744
7,134,0.123275
4,42,0.038638
8,14,0.012879
3,8,0.007360


# Std solution
- A validação cruzada será realizada com split estratificado na "quality" propositadamente para se ter um treinamento sobre os mesmos dados dos modelos binários
- As random states garantem repetibilidade.

In [5]:
def calcular_estatisticas(lista, output=True):
    
    mean = round(np.mean(lista), 4)
    std = round(np.std(lista), 4)
    min = round(np.min(lista), 4)
    max = round(np.max(lista), 4)
    
    if output:
        output=f"Média da validação cruzada (std): {mean} ({std})\n(Min/Máx): ({min}/{max})"
        print(output)
    
    return mean, std, min, max

In [6]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=2025)
rmse_list = []
for train_index_fold, val_index_fold in skf.split(df_train, df_train['quality']):
    df_train_fold, df_val_fold = df_train.loc[train_index_fold], df_train.loc[val_index_fold]
    
    X_fold = df_train_fold[feature_columns]
    y_fold = df_train_fold['quality']
    X_val = df_val_fold[feature_columns]
    y_val = df_val_fold['quality']
    
    model = LGBMRegressor(verbose=-1, random_state=2025)
    model.fit(X_fold, y_fold)
    y_val_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_val_pred)
    rmse = np.sqrt(mse)
    rmse_list.append(rmse)
    
# Calcular média, desvio padrão, mínimo e máximo
print('RMSE')
mean, std, min, max = calcular_estatisticas(rmse_list)

RMSE
Média da validação cruzada (std): 0.6756 (0.049)
(Min/Máx): (0.6097/0.7334)


# Otimização de hiperparâmetros

In [7]:
space = [
    Real(low=1e-4, high=3e-1, prior='log-uniform'),  # learning_rate     = params[0]
    Integer(low=2, high=256),                        # num_leaves        = params[1]
    Integer(low=1, high=100),                        # min_child_samples = params[2]
    Integer(low=2, high=20),                         # max_depth         = params[3]
    Real(low=0.5, high=1.0),                         # subsample         = params[4]
    Real(low=0.5, high=1.0),                         # colsample_bytree  = params[5]
    Integer(low=100, high=1000),                     # n_estimators      = params[6]
    Real(low=0.0, high=1.0),                         # reg_alpha         = params[7]
    Real(low=0.0, high=1.0),                         # reg_lambda        = params[8]
]



def get_model(params):

    model = LGBMRegressor(
        learning_rate       = params[0],
        num_leaves          = params[1],
        min_child_samples   = params[2],
        max_depth           = params[3],
        subsample           = params[4],
        colsample_bytree    = params[5],
        n_estimators        = params[6],
        reg_alpha           = params[7],
        reg_lambda          = params[8],
        subsample_freq      = 1,
        random_state        = 2025
    )
    
    return model


def objective_minimize(params):
    rmse_list = []
    for train_index_fold, val_index_fold in skf.split(df_train, df_train['quality']):
        df_train_fold = df_train.iloc[train_index_fold]
        df_val_fold = df_train.iloc[val_index_fold]
        
        X_fold = df_train_fold[feature_columns]
        y_fold = df_train_fold['quality'] 
        X_val = df_val_fold[feature_columns]
        y_val = df_val_fold['quality']
        
        model = get_model(params)
        model.fit(X_fold, y_fold)
        y_val_pred = model.predict(X_val)
    
        mse = mean_squared_error(y_val, y_val_pred)
        rmse = np.sqrt(mse)
        rmse_list.append(rmse)
    metric = np.mean(rmse_list)
    return metric  # Queremos minimizar o RMSE

In [8]:
# Otimização de (hiper)parametros
result = gp_minimize(objective_minimize, space, random_state=2025, n_calls=100, n_random_starts=30)

# Melhores parâmetros
best_params = result.x
print(best_params)
print(result.fun)

[0.004487702458075282, np.int64(256), np.int64(17), np.int64(6), 0.8043979446328742, 1.0, np.int64(861), 0.7127967610834102, 0.0]
0.640807757816656


# Escolha e avaliação do modelo
- Primeiramente deve-se avaliar o desempenho médio e desvios para decidir se a configuração utilizada produz a consistencia desejada
- Verificar se há melhoria em comparação a outras estratégias
- Por fim, deve-se especificar o modelo. Dentre os processos de escolha, é comum:
    - Dado os melhores parâmetros, utilizar o modelo de melhor desempenho nas split folds da validação cruzada
    - Realizar o "Reffit" do modelo, isto é, com os melhores hiperparâmetros treinar um novo modelo com a totalidade dos dados de test_index
- Como se tem uma amostragem reduzida da classe minoritária, optou-se pela segunda abordagem, visto que nela um número maior de dados é utilizado em treinamento
    - Mesmo assim, a validação cruzada com os "melhores parâmetros" será realizada para a comparação com a avaliação em modo padrão, realizada anteriormente.

In [13]:
# Validação cruzada
rmse_list = []
for train_index_fold, val_index_fold in skf.split(df_train, df_train['quality']):
    df_train_fold, df_val_fold = df_train.loc[train_index_fold], df_train.loc[val_index_fold]
    X_fold = df_train_fold[feature_columns]
    y_fold = df_train_fold['quality']
    X_val = df_val_fold[feature_columns]
    y_val = df_val_fold['quality']
    
    model = get_model(best_params)
    model.fit(X_fold, y_fold)
    y_val_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_val_pred)
    rmse = np.sqrt(mse)
    rmse_list.append(rmse)
    
# Calcular média, desvio padrão, mínimo e máximo
print('RMSE')
mean, std, min, max = calcular_estatisticas(rmse_list)

RMSE
Média da validação cruzada (std): 0.6408 (0.0383)
(Min/Máx): (0.5908/0.6829)


In [14]:
# Modelo final
X_train = df_train[feature_columns]
y_train = df_train['quality']

model = get_model(best_params)
model.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.004487702458075282, max_depth=np.int64(6),
              min_child_samples=np.int64(17), n_estimators=np.int64(861),
              num_leaves=np.int64(256), random_state=2025,
              reg_alpha=0.7127967610834102, subsample=0.8043979446328742,
              subsample_freq=1)

# Simulação de desempenho em produção/blind set

In [15]:
df_test = pd.read_csv('../data/df_test.csv')
df_test.head(2)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,6.8,0.61,0.20,1.8,0.077,11.0,65.0,0.99710,3.54,0.58,9.3,5
1,11.3,0.36,0.66,2.4,0.123,3.0,8.0,0.99642,3.20,0.53,11.9,6


In [16]:
X_test = df_test.drop(columns=['quality'])
y_test = df_test['quality']

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)

print(f'RSME: {rmse}')

RSME: 0.40131437152953814


# Save Model

In [17]:
path_model = '../ml_models/regression_model.pkl'
with open(path_model, 'wb') as arquivo:
    pickle.dump(model, arquivo)

# Test loaded model

In [18]:
with open(path_model, 'rb') as arquivo:
    model = pickle.load(arquivo)

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)

print(f'RSME: {rmse}')

RSME: 0.40131437152953814


# Julio Patti Pereira 
- 14/02/2025